In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd
import psycopg2
from geoalchemy2 import Geometry, WKTElement
from sqlalchemy import *
from shapely.geometry import MultiPolygon
from zipfile import ZipFile
import requests 
import sys

In [2]:
import yaml

with open('../../config/postgres.yaml') as f:
    engine_configs = yaml.load(f, Loader=yaml.FullLoader)
    
try:
    engine = create_engine('postgresql://{username}:{password}@{host}:{port}/{dbname}'.format(**engine_configs))
except Exception as e:
    print("Uh oh, can't connect. Invalid dbname, user or password?")
    print(e)

In [3]:
def process_geometry_SQL_insert(gdf):
    gdf['geom'] = gdf['geometry'].apply(lambda x: WKTElement((MultiPolygon([x]) if x.geom_type == 'Polygon' else x).wkt, srid=4326))
    gdf = gdf.drop('geometry', 1)
    return gdf

In [4]:
CITY='LA'

## Neighborhoods

In [5]:
sql = """INSERT INTO spatial_groups (city, core_geom, core_id, lower_ids, spatial_name, approx_geom)
SELECT a.city, a.core_geom, a.core_id, array_agg(a.core_id), 'core', ST_multi(a.core_geom)
FROM spatial_groups a
where a.city='{city}' and a.spatial_name = 'ego'
GROUP BY a.core_id, a.core_geom, a.city;
""".format(city=CITY, tempname=CITY.lower())

result = engine.execute(text(sql))

## Land use

ref: http://dts.edatatrace.com/dts3/content/doc/whelp/mergedProjects/dts2tt/mergedProjects/dts2ttcs/land_use_la.htm

In [7]:
land_gdf = gpd.read_file('zip://../../data/LA/land_use/Parcels 2014 Tax Roll.zip')
#land_gdf = land_gdf[(~(land_gdf['geometry'].isnull())) & (~(land_gdf['UseCode'].isnull()))]
land_gdf = land_gdf.drop_duplicates(subset=['ain'])

#land_gdf = land_gdf.rename(columns={'SQFTmain': 'sqftmain', 'UseCode': 'usecode', 'YearBuilt': 'yearbuilt', 'Roll_totLa': 'value'})

#land_gdf = land_gdf[['AssessorID', 'sqftmain', 'usecode', 'geometry', 'value']]

land_gdf = land_gdf[['ain', 'geometry']]

land_gdf.head()

,ain,geometry
0,7024005002,"POLYGON ((-118.06987 33.86189, -118.06987 33.8..."
1,7024008023,"POLYGON ((-118.06811 33.86152, -118.06830 33.8..."
2,7024001042,"POLYGON ((-118.06854 33.86550, -118.06857 33.8..."
3,7024005004,"POLYGON ((-118.06987 33.86160, -118.07020 33.8..."
4,7024006008,"POLYGON ((-118.07091 33.85923, -118.07125 33.8..."


In [8]:
zip_file = ZipFile('../../data/LA/land_use/parcels_data_2013.csv.zip')
land_2013_df = pd.read_csv(zip_file.open('parcels_data_2013.csv'), dtype={'AIN': str})
land_2013_df = land_2013_df.rename(columns={'SQFTmain': 'sqftmain', 
                                            'AIN': 'ain',
                                            'PropertyUseCode': 'usecode', 
                                            'YearBuilt': 'yearbuilt', 'TotalValue': 'value'})
land_2013_df = land_2013_df[['ain', 'sqftmain', 'usecode', 'value']]
land_2013_df.head()

/home/denadai/.local/share/virtualenvs/crime-environment-u9RldRjF/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (37) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,ain,sqftmain,usecode,value
0,5653015020,1498,0101,458864.0
1,5653015021,1532,0100,363890.0
2,5653015022,1186,0101,570000.0
3,5653015023,2192,0101,618177.0
4,5653015024,2108,0100,505364.0


In [9]:
land_2013_df = land_2013_df.drop_duplicates(subset=['ain'])

In [10]:
land_gdf = pd.merge(land_gdf, land_2013_df, on='ain')
print(len(land_gdf))

2378363


In [11]:
wrong_pids = ['4211017901', '4211017804', '4218005900', '4221031008',
       '4218020900', '4224013902', '2109001903', '2678027900',
       '2679025900', '2680020901', '2687017900', '2688030900',
       '2707003011', '2708010900', '2726009901', '2726012900',
       '2746010042', '2746013901', '2761001906', '2779016900',
       '2779047900', '2784003801', '2779010900', '2708021001',
       '2111029903', '4211016902', '4211015904', '4211007916',
       '2786002902', '2727021907', '4211014800', '4211017805',
       '4218005902', '2108025900', '2678020900', '2687023012',
       '2687020903', '2688024901', '2688031900', '2786002901',
       '2708010013', '2708020005', '2726010900', '2761030904',
       '2779017900', '2780005900', '2138014904', '2783028902',
       '4211014902', '4211017807', '4224013901', '2108026900',
       '2109001902', '2113006900', '2677016900', '2679016901',
       '2685019900', '2689016901', '2688043900', '2786002813',
       '2726014900', '2761032900', '2770018808', '2780004900',
       '2681011902', '2111029902', '2779005900', '4218005901',
       '2680018902', '2707003005', '2708020001', '2707002004',
       '2761001907', '4211016901', '4211015902', '4211007917',
       '2148032902', '4211007919', '4211014904', '4211017900',
       '4211017803', '4211014901', '2108031900', '2685013032',
       '2685013031', '2686003008', '2685023030', '2685018900',
       '2685013900', '2689017900', '2708020012', '2746005900',
       '2748001803', '2761031902', '2761040901', '2770018904',
       '2770018903', '2779010901', '2779011905', '2779020905',
       '2111029901', '4221022176', '2761001814', '4211007012',
       '4224013900', '2783028801', '2689019900', '2205008901',
       '2231018901', '2225010902', '2226017901', '2231002909',
       '2231017900', '2205007900', '4211017901', '4211017804', '4218005900', '4221031008',
       '4218020900', '4224013902', '5409013910', '5410015826',
       '2109001903', '2678027900', '2679025900', '2680020901',
       '2687017900', '2688030900', '2707003011', '2708010900',
       '2726009901', '2726012900', '2746010042', '2746013901',
       '2761001906', '2779016900', '2779047900', '2784003801',
       '5173021811', '5173020911', '5173023900', '5173020903',
       '2779010900', '2708021001', '5170011803', '2111029903',
       '4211016902', '4211015904', '4211007916', '5172014806',
       '5172014901', '2786002902', '2727021907', '4211014800',
       '4211017805', '4218005902', '5409013905', '5409013906',
       '5409015922', '5409014904', '5409021903', '5409019903',
       '2108025900', '2678020900', '2687023012', '2687020903',
       '2688024901', '2688031900', '2786002901', '2708010013',
       '2708020005', '2726010900', '2761030904', '2779017900',
       '2780005900', '5171024910', '5173020902', '5173020901',
       '5173023901', '2138014904', '5164004804', '5172013010',
       '5172013002', '5164004902', '2783028902', '4211014902',
       '4211017807', '4224013901', '5409020910', '5409020911',
       '5447017902', '2108026900', '2109001902', '2113006900',
       '2677016900', '2679016901', '2685019900', '2689016901',
       '2688043900', '2786002813', '2726014900', '2761032900',
       '2770018808', '2780004900', '5171024010', '2681011902',
       '2111029902', '2779005900', '4218005901', '2680018902',
       '2707003005', '2708020001', '2707002004', '5166001901',
       '5164017906', '2761001907', '5173022902', '4211016901',
       '4211015902', '4211007917', '5172013803', '5172013901',
       '2148032902', '4211007919', '4211014904', '5171015901',
       '4211017900', '4211017803', '4211014901', '5409014905',
       '2108031900', '2685013032', '2685013031', '2686003008',
       '2685023030', '2685018900', '2685013900', '2689017900',
       '2708020012', '2746005900', '2748001803', '2761031902',
       '2761040901', '2770018904', '2770018903', '2779010901',
       '2779011905', '2779020905', '5170010805', '5164004901',
       '5173021902', '5173023902', '5173020810', '5173021810',
       '5173021904', '5173023805', '2111029901', '4221022176',
       '5171014808', '2761001814', '5173022808', '5173022903',
       '5173022901', '4211007012', '4224013900', '2783028801',
       '2689019900', '5173024900', '5166001900', '2205008901',
       '2231018901', '2225010902', '2226017901', '2231002909',
       '2231017900', '2205007900', '5447032900', '2368001030', '2366035901', '2366036905',
       '2367015900', '2367018900', '2368019900', '2368023900',
       '2375018903', '2126038901', '2134016901', '2134024904',
       '2136015905', '2136017904', '2137013900', '2137014900',
       '2137015902', '2137012900', '2138006901', '2138022901',
       '2123022901', '5435038027', '5435038902', '5435039903',
       '5437028903', '5437028906', '5437034908', '5437028907',
       '5437035901', '5437034909', '5437034904', '5442010901',
       '5442010902', '5442002916', '5445011042', '5445005904',
       '5445006901', '5445007900', '5445010903', '5445006905',
       '5168023015', '5169029013', '5168016904', '5170010900',
       '2424042901', '2138014906', '5593012909', '5168023902',
       '5171015900', '5437035902', '5172013900', '5593001270',
       '5442009902', '2126033900', '5593018907', '5410002900',
       '2360002909', '2366033900', '2366033901', '2368007901',
       '2375021903', '2126038005', '2127011904', '2128031901',
       '2138006903', '2138011900', '2138014905', '5435039006',
       '5415002900', '5435036900', '5437028904', '5442002915',
       '5437028900', '5445006903', '5445007901', '5169029272',
       '5169029012', '5169016902', '5581003017', '5581003021',
       '5581004023', '5593001902', '5169028017', '5435038904',
       '2360003913', '5593002916', '5445004001', '5445004900',
       '5447027901', '5415003901', '5415003900', '2360014902',
       '2366020903', '2366027902', '2375004900', '2128003901',
       '2138011902', '2124018906', '5442010020', '5442002903',
       '5445008908', '5445007902', '5169029010', '5169015901',
       '2423030906', '2423031902', '2423035902', '5581003011',
       '5593018900', '5169016011', '5169029902', '5168017900',
       '5435037904', '2131010900', '5442009900', '2127001903',
       '5410006900', '2360012900', '2366026902', '2367018901',
       '2375019903', '2132009900', '2138017900', '2138017901',
       '2138023900', '2138029902', '2123021900', '2124001905',
       '5437029900', '5435039900', '5445011043', '5445012044',
       '5437028902', '5437036902', '5445008907', '5168016002',
       '5168016903', '5581003008', '5581004022', '5593001903',
       '5171014900', '5172014900', '5593002907', '5168017902',
       '5593001900', '5445004902', '5445005903', '5445002902',
       '5593001901', '2127005900', '2368010902', '5173024900',
       '2248029903', '2263020902', '2263016904', '2248001904',
       '2248028906', '2263013902', '2263015902', '2263021902',
       '2248001905', '2263024900', '2263014902', '2271001902', 
             '5173021811', '5173020911', '5173023900', '5171014809',
       '5171015900', '5172013900', '5173023901', '5164004804',
       '5164004902', '5173020910', '5173021903', '5173022902',
       '5171014900', '5173020907', '5173021902', '5173023902',
       '5172014900', '5173020810', '5173021810', '5173021904',
       '5173023805', '5173022808', '5173022903', '5173022901',
       '5173024900']
land_gdf = land_gdf[~land_gdf.ain.isin(wrong_pids)]

In [12]:
land_gdf['landuse'] = 'none'

land_gdf.loc[land_gdf['usecode'].str[:1] == '0', 'landuse'] = 'residential'

land_gdf.loc[(land_gdf['usecode'].str[:1].isin({'1', '2', '3', '4', '5', '7'})) & (~land_gdf['usecode'].isin({'7100', '8840'})), 'landuse'] = 'commercial'
land_gdf.loc[land_gdf['usecode'].str[:2].isin({'82', '83'}), 'landuse'] = 'commercial'
land_gdf.loc[land_gdf['usecode'].isin({'8820', '8000', '8821', '8822', '8823', '8824', '8825', '8826', '8827', '8828', '8829', '8830', '8831', '8832', '8833', '8834', '8835', '8855', '8861', '8862', '8863', '8864', '8865', '8872', '8873', '8874', '8800', '8890', '8900'}), 'landuse'] = 'commercial'

land_gdf.loc[land_gdf['usecode'].str[:1] == '6', 'landuse'] = 'recreational'
land_gdf.loc[land_gdf['usecode'].isin({'7100', '8840', '8840', '8841', '8842', '8843', '8844', '8845', '8847', '8848', '8849', '8851', '8852', '8853'}), 'landuse'] = 'recreational'

# Vacant
land_gdf.loc[land_gdf['usecode'].str[-1] == 'V', 'landuse'] = 'vacant'

#Fixes
land_gdf.loc[land_gdf['usecode'].isin({'8100', '8109', '810X', '8860', '8500'}), 'landuse'] = 'none'

In [13]:
land_gdf.loc[land_gdf.ain.isin(['7467032900', '7469018904', '7469030901', '7469030900',
       '7563001901', '7563001900', '7563002908', '7563002914',
       '6038013900', '5414020901', '5414020900', '2178007900',
       '2184026901', '5666025907', '6049025901', '4432001903',
       '4432005913', '4432005800', '4432006901', '4490011903',
       '4493014900', '4422003900', '4432002918', '4432002924',
       '4432002923', '4434001903', '5037027915', '5046013900',
       '5160001901', '5512004903', '5630030908', '4370012902',
       '4387002900', '5404014900', '5581011900', '5581012900',
       '5581010900', '5581013901', '5583025900', '5593002908',
       '5593002910', '5109022900', '5161004909', '2526004901',
       '2526004900', '2552007902', '2569021900', '5029017905',
       '4355012904', '5029020904', '2701001910', '4432002919',
       '7412014900', '7560028900', '2384024900', '5029017927',
       '5459004930', '7446001901', '7467025900', '7469028900',
       '5414027900', '2177034902', '2177034901', '5666024901',
       '2184005900', '4432001900', '4491006900', '4409001902',
       '4409001900', '4422002900', '4432002920', '4432003904',
       '5028004902', '5029017921', '5029017910', '2470002901',
       '2546013903', '2545022900', '4387002904', '4387017906',
       '4387016900', '5565005900', '5565004900', '5570021902',
       '5415004900', '5415012902', '5577019901', '5581016900',
       '5101002900', '2551012901', '2846003900', '7563002909',
       '5029017926', '5029017919', '5593002906', '2701001912',
       '4493014906', '5581007912', '4379029900', '4379028902',
       '4431009901', '4432003906', '5211021901', '2872001900',
       '4386003900', '4386005900', '2177034900', '7467031900',
       '7469018902', '7563002913', '7563002906', '7412012900',
       '2184028901', '2184026902', '2184028900', '5672021900',
       '6049025900', '6070004900', '4432005915', '4432006902',
       '4432005914', '4432001901', '4490010900', '4490011902',
       '4490024900', '4491009900', '4434001900', '4432002922',
       '4434001901', '4432002925', '4432002917', '5037028905',
       '5037028902', '2470001905', '2470002900', '2545024901',
       '5608001902', '5630030906', '5630030907', '4379027902',
       '4379027903', '4379027900', '4380034902', '4387002905',
       '5415012901', '5581017900', '5581014900', '5581026900',
       '5149031900', '5161005916', '5869016900', '4434001902',
       '5029017900', '5404015900', '5029020905', '2701001917',
       '5415005906', '5593002905', '4357004901', '5577011902',
       '7561025902', '5593002909', '2701002909', '4493015900',
       '2382015900', '4432001902', '7422017900', '7469018903',
       '7469029900', '7563001902', '7412012903', '7562021900',
       '7563006902', '2180024900', '2184005901', '2184027901',
       '2671001903', '4490011900', '4490017900', '4491001900',
       '4409001904', '4434005900', '4432002921', '5037028908',
       '5160001900', '4370012901', '4386008901', '4386015906',
       '4387002902', '5415004902', '5415005905', '5415005902',
       '5415012903', '5582001900', '5593002912', '5113008912',
       '5161005923', '2526003909', '2526003910', '2551003900',
       '2551015902', '2551012900', '2552004900', '2552007906',
       '2552009902', '2552007907', '2553017900', '2569022901',
       '5415005904', '5047014900', '5029017924', '5581008900',
       '5029017911', '5593002907', '4382029900', '4431009900',
       '7412015900', '5593002917', '4432004901', '2552005901',
       '7412010903', '4386004902', '2177034904', '2180026900',
       '2180025900', '5302002900', '5302006900', '2287009903',
       '2287009902', '2287009901', '2292014901', '2292014900',
       '2292013901', '5630030902', '5302001900', '2820019900',
       '5303025901']), 'landuse'] = 'recreational'


In [14]:
land_gdf = land_gdf.reset_index()
land_gdf.head()

,index,ain,geometry,sqftmain,usecode,value,landuse
0,0,7024005002,"POLYGON ((-118.06987 33.86189, -118.06987 33.8...",3033,0100,248617.0,residential
1,1,7024008023,"POLYGON ((-118.06811 33.86152, -118.06830 33.8...",2036,0101,487044.0,residential
2,2,7024001042,"POLYGON ((-118.06854 33.86550, -118.06857 33.8...",950,0100,72176.0,residential
3,3,7024005004,"POLYGON ((-118.06987 33.86160, -118.07020 33.8...",2500,0100,415616.0,residential
4,4,7024006008,"POLYGON ((-118.07091 33.85923, -118.07125 33.8...",2605,0100,504556.0,residential


## Net area

In [15]:
unique_land_gdf = land_gdf.copy()
unique_land_gdf.loc[:, 'x'] = unique_land_gdf.geometry.centroid.x
unique_land_gdf.loc[:, 'y'] = unique_land_gdf.geometry.centroid.y
unique_land_gdf = unique_land_gdf.drop_duplicates(subset=['x', 'y'])

In [16]:
ins_gdf = process_geometry_SQL_insert(unique_land_gdf)
ins_gdf.to_sql('temptable_{}'.format(CITY.lower()), engine, if_exists='replace', index=False, dtype={'geom': Geometry('MultiPolygon', srid=4326)})

In [17]:
sql = """
UPDATE temptable_{tempname} p SET geom=ST_Multi(ST_buffer(p.geom, 0.0)) 
WHERE NOT ST_Isvalid(p.geom)
""".format(city=CITY, tempname=CITY.lower())

result = engine.execute(text(sql))

In [18]:
sql = """
DELETE 
FROM temptable_{tempname} t
USING unused_areas u 
WHERE u.city = '{city}' AND ST_Intersects(u.geom, t.geom) AND (NOT ST_Touches(u.geom, t.geom)) 
AND (ST_Contains(u.geom, t.geom) OR ST_AREA(ST_Intersection(t.geom, u.geom))/ST_Area(t.geom) > 0.5);
""".format(city=CITY, tempname=CITY.lower())

result = engine.execute(text(sql))

In [19]:
sql = """
INSERT INTO spatial_groups_net_area (sp_id, city, spatial_name, used_area) 
SELECT sp_id, city, spatial_name, SUM(ST_Area(ST_Intersection(s.approx_geom, t.geom)::geography))/1000000.
FROM temptable_{tempname} t
INNER JOIN spatial_groups s ON ST_Intersects(s.approx_geom, t.geom) AND NOT ST_Touches(s.approx_geom, t.geom)
WHERE s.city = '{city}' AND s.spatial_name='core'
GROUP BY sp_id, city, spatial_name;
""".format(city=CITY, tempname=CITY.lower())

result = engine.execute(text(sql))

## Refresh materialized views

In [20]:
sql = """
REFRESH MATERIALIZED VIEW spatial_groups_unused_areas;
"""

result = engine.execute(text(sql))